# P0 with Exception Test Cases

#### Author: Kevin Zhou, Fanping Jiang, Meijing Li

In [1]:
import nbimporter; nbimporter.options["only_defs"] = False
from IPython.display import display
from P0 import compileString
from ST import printSymTab

def runwasm(wasmfile):
    from IPython.core.display import display, Javascript
    display(Javascript("""
    const params = { 
        P0lib: { 
            write: i => this.append_stream({text: '' + i, name: 'stdout'}),
            writeln: () => this.append_stream({text: '\\n', name: 'stdout'}),
            read: () => window.prompt()
        }
    }

    fetch('""" + wasmfile + """') // asynchronously fetch file, return Response object
      .then(response => response.arrayBuffer()) // read the response to completion and stores it in an ArrayBuffer
      .then(code => WebAssembly.compile(code)) // compile (sharable) code.wasm
      .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
    // .then(instance => instance.exports.program()); // run the main program; not needed if start function specified
    """))

def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i)
    def writeln(s): print('\n')
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

    
from wasmer import engine, Store, Module, Instance, ImportObject, Function
from wasmer_compiler_cranelift import Compiler

def runwasmer(wasmfile):
    def write(i: int): print(i)
    def writeln(): print('\n')
    def read() -> int: return int(input()) 
    store = Store(engine.JIT(Compiler))
    module = Module(store, open(wasmfile, 'rb').read())
    import_object = ImportObject()
    import_object.register("P0lib", {"write": Function(store, write),
                                     "writeln": Function(store, writeln),"read": Function(store, read)})
    instance = Instance(module, import_object)

In [2]:
import warnings
warnings.filterwarnings("ignore")

## Tests for Explicit Exceptions

### Square Root Calculator

- Calculate the square root of `-2`

    Since `-2` is smaller than 0, `throw 28` takes affect. The program outputs `123` as expected.

    Run the cell below to see the generated WebAssembly code.

In [3]:
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw 28 else 
        r := 1
        while (r × r) ≤ x do
            r := r + 1
        r := r - 1
program equationsolver
    var a: integer
    try
        a ← sqrt(-2)
        write(a)
    catch 28
        write(123)
""")

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(tag $e28 (param i32))
(func $sqrt (param $x i32) (result i32)
(local $r i32)
(local $0 i32)
local.get $x
i32.const 0
i32.lt_s
if
i32.const 28
throw $e28
else
i32.const 1
local.set $r
loop
local.get $r
local.get $r
i32.mul
local.get $x
i32.le_s
if
local.get $r
i32.const 1
i32.add
local.set $r
br 1
end
end
local.get $r
i32.const 1
i32.sub
local.set $r
end
local.get $r
)
(func $program
(local $a i32)
(local $0 i32)
try
i32.const -2
call $sqrt
local.set $a
local.get $a
call $write
catch $e28
drop
i32.const 123
call $write
catch_all
end
)
(memory 1)
(start $program)
)


Then run the cell below to execute the generated WebAssembly code.

In [4]:
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw 28 else 
        r := 1
        while (r × r) ≤ x do
            r := r + 1
        r := r - 1
program equationsolver
    var a: integer
    try
        a ← sqrt(-2)
        write(a)
    catch 28
        write(123)
""", 'test.wat')

In [5]:
!wat2wasm --enable-exceptions test.wat

In [6]:
runwasm('test.wasm')

<IPython.core.display.Javascript object>

- Since the exception tag is up to the user to define, it can be any positive integers, as long as the tag in throw matches the tag in catch.

    Now we change the `throw 28` to `throw 42` as well as `catch 28` to `catch 42`, and run the cells below. You should get the same result.

In [7]:
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw 42 else 
        r := 1
        while (r × r) ≤ x do
            r := r + 1
        r := r - 1
program equationsolver
    var a: integer
    try
        a ← sqrt(-2)
        write(a)
    catch 42
        write(123)
""", 'test.wat')

In [8]:
!wat2wasm --enable-exceptions test.wat

In [9]:
runwasm('test.wasm')

<IPython.core.display.Javascript object>

- You have to match the exception tag in order to catch correctly, otherwise it will not catch as expected.

    Now if we `throw 42` but `catch 20`, there is a mismatch, so it will print nothing.

In [10]:
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw 42 else 
        r := 1
        while (r × r) ≤ x do
            r := r + 1
        r := r - 1
program equationsolver
    var a: integer
    try
        a ← sqrt(-2)
        write(a)
    catch 20
        write(123)
""", 'test.wat')

In [11]:
!wat2wasm --enable-exceptions test.wat

In [12]:
runwasm('test.wasm')

<IPython.core.display.Javascript object>

- Finally, we calculate the square root of `81`

    Since `81` is greater than 0, it outputs the square root of `81` which is `9` as expected.

In [13]:
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw 28 else 
        r := 1
        while (r × r) ≤ x do
            r := r + 1
        r := r - 1
program equationsolver
    var a: integer
    try
        a ← sqrt(81)
        write(a)
    catch 28
        write(123)
""", 'test.wat')

In [14]:
!wat2wasm --enable-exceptions test.wat

In [15]:
runwasm('test.wasm')

<IPython.core.display.Javascript object>

### Teting for No Catch

Based on the grammar for try-catch statement `'try' statementSuite {'catch' i statementSuite}`, there could be no 'catch' block after the 'try' block.

The test case below is for showing the ability of 0 catch statement:

The program prints the input value if the input is greater or equal to 0; otherwise, no catch block exists to capture the exception therefore it prints nothing.

In [21]:
compileString("""
procedure nonNeg(x: integer) → (r: integer)
    if x < 0 then throw 1
    else r := x 
program noCatch
    var x, a: integer
    try
        x ← read()
        a ← nonNeg(x)
        write(a)
""", 'test.wat')

In [23]:
!wat2wasm --enable-exceptions test.wat

In [25]:
runwasm('test.wasm')

<IPython.core.display.Javascript object>

### Multiple Catch Testing

The test cases below is for showing the ability of multiple `catch` application.

The follwing program prints `-1` if it is a negative integer and prints `1` if it is a positive integer. It prints out `0` if the input number is `0`.



Run the cell below to see the generated WebAssembly code.

In [ ]:
compileString("""
procedure posOrNeg(x: integer) → (r: integer)
    if x < 0 then throw 65536
    if x > 0 then throw 65535
    else r := x 
program positiveOrNegative
    var x, a: integer
    try
        x ← read()
        a ← posOrNeg(x)
        write(a)
    catch 65536
        write(-1)
    catch 65535
        write(1)
""")

Then run the cell below to execute the generated WebAssembly code.

In [26]:
compileString("""
procedure posOrNeg(x: integer) → (r: integer)
    if x < 0 then throw 65536
    if x > 0 then throw 65535
    else r := x 
program positiveOrNegative
    var x, a: integer
    try
        x ← read()
        a ← posOrNeg(x)
        write(a)
    catch 65536
        write(-1)
    catch 65535
        write(1)
""", 'test.wat')

In [27]:
!wat2wasm --enable-exceptions test.wat

In [28]:
runwasm('test.wasm')

<IPython.core.display.Javascript object>

## Implicit Exception Test Cases

### Division by 0 

The zero division exception is built-in with the exception tag `111`, the below `P0` code is trying to calculate `99/(34*5-170)`, which is `99/0`. It should trigger `throw 111` thus the statement under `catch 111` will be executed. The output is `111`.


Run the cell below to see the generated WebAssembly code.

In [29]:
# logic: if a == 0 then throw 111 else b := (99 div a); write(b)
compileString("""
program DividedByZero
    var a, b: integer
        try
            a := (34 × 5 - 170)
            b := (99 div a)
            write(b)
        catch 111
            write(111)
""")

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(tag $e111 (param i32))
(func $program
(local $a i32)
(local $b i32)
(local $0 i32)
try
i32.const 0
local.set $a
local.get $a
i32.const 0
i32.eq
if
i32.const 111
throw $e111
else
i32.const 99
local.get $a
i32.div_s
local.set $b
local.get $b
call $write
end
catch $e111
drop
i32.const 111
call $write
catch_all
end
)
(memory 1)
(start $program)
)


Then run the cell below to execute the generated WebAssembly code. The expected output is `111`

In [30]:
compileString("""
program DividedByZero
    var a, b: integer
        try
            a := (34 × 5 - 170)
            b := (99 div a)
            write(b)
        catch 111
            write(111)
""", 'test.wat')

In [31]:
!wat2wasm --enable-exceptions test.wat

In [32]:
runwasm("test.wasm")

<IPython.core.display.Javascript object>

Now we trying to calculate `99/(34*5-170+3)`, which is `99/3`, and it should output `33` instead of raising exception.

Then run the cell below to execute the generated WebAssembly code.

In [33]:
compileString("""
program DividedByZero
    var a, b: integer
        try
            a := (34 × 5 - 170 + 3)
            b := (99 div a)
            write(b)
        catch 111
            write(111)
""", 'test.wat')

In [34]:
!wat2wasm --enable-exceptions test.wat

In [35]:
runwasm("test.wasm")

<IPython.core.display.Javascript object>

### Mod by 0

- The same logic goes under the `Mod by 0` exception, with the only difference of the exception tag as `112`.

    The code below is trying to get `99 mod (34*5-170)`, which is `99 mod 0`, it should trigger implicit `throw 112` thus the statement under `catch 112` will be executed. The output is `112`.

Run the cell below to see the generated WebAssembly code.

In [36]:
# logic: if a == 0 then throw 112 else b := (99 mod a); write(b)
compileString("""
program ModByZero
    var a, b: integer
        try
            a := (34 × 5 - 170)
            b := (99 mod a)
            write(b)
        catch 112
            write(112)
""")

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(tag $e112 (param i32))
(func $program
(local $a i32)
(local $b i32)
(local $0 i32)
try
i32.const 0
local.set $a
local.get $a
i32.const 0
i32.eq
if
i32.const 112
throw $e112
else
i32.const 99
local.get $a
i32.rem_s
local.set $b
local.get $b
call $write
end
catch $e112
drop
i32.const 112
call $write
catch_all
end
)
(memory 1)
(start $program)
)


Then run the cell below to execute the generated WebAssembly code. The expected output is `112`.

In [37]:
compileString("""
program ModByZero
    var a, b: integer
        try
            a := (34 × 5 - 170)
            b := (99 mod a)
            write(b)
        catch 112
            write(112)
""", 'test.wat')

In [38]:
!wat2wasm --enable-exceptions test.wat

In [39]:
runwasm("test.wasm")

<IPython.core.display.Javascript object>

Now we trying to calculate the remainder of `99/(34*5-170+5)`, which is `99/5`, and it should output `4` instead of raising exception.

In [40]:
compileString("""
program ModByZero
    var a, b: integer
        try
            a := (34 × 5 - 170 + 5)
            b := (99 mod a)
            write(b)
        catch 112
            write(112)
""", 'test.wat')

In [41]:
!wat2wasm --enable-exceptions test.wat

In [42]:
runwasm("test.wasm")

<IPython.core.display.Javascript object>

### Index Out of Bounds Exception

The `index out of bounds` exception is built-in with the exception tag of `110`. Below is an example.


Since the program is trying to access the index that is out of the bound, `throw 110` will be triggered and thus the statement under `catch 110` will be executed, therefore the output is `110`.

Run the cell below to see the generated WebAssembly code.

In [43]:
# logic: if (i < x.tp.lower) or (i >= x.tp.lower + x.tp.length) then throw else write(x[i])
compileString("""
var x: [2 .. 4] → integer
program IndexOutOfBounds
    var i: integer
    x[2] := 2; x[3] := 3; x[4] := 4
    try
        i := 5
        write(x[i])
    catch 110
        write(110)
""")

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(tag $e110 (param i32))
(func $program
(local $i i32)
(local $0 i32)
i32.const 0
i32.const 2
i32.store
i32.const 4
i32.const 3
i32.store
i32.const 8
i32.const 4
i32.store
try
i32.const 5
local.set $i
local.get $i
i32.const 2
i32.lt_s
if (result i32)
i32.const 1
else
local.get $i
i32.const 5
i32.ge_s
end
if
i32.const 110
throw $e110
else
local.get $i
i32.const 2
i32.sub
i32.const 4
i32.mul
i32.const 0
i32.add
i32.load
call $write
end
catch $e110
drop
i32.const 110
call $write
catch_all
end
)
(memory 1)
(start $program)
)


Then run the cell below to execute the generated WebAssembly code.

In [44]:
compileString("""
var x: [2 .. 4] → integer
program IndexOutOfBounds
    var i: integer
    x[2] := 2; x[3] := 3; x[4] := 4
    try
        i := 5
        write(x[i])
    catch 110
        write(110)
""", 'test.wat')

In [45]:
!wat2wasm --enable-exceptions test.wat

In [46]:
runwasm("test.wasm")

<IPython.core.display.Javascript object>

What if the index is negative? The negative index is also out of bound, thus the output is still `110`.

In [47]:
compileString("""
var x: [2 .. 4] → integer
program IndexOutOfBounds
    var i: integer
    x[2] := 2; x[3] := 3; x[4] := 4
    try
        i := -5
        write(x[i])
    catch 110
        write(110)
""", 'test.wat')

In [48]:
!wat2wasm --enable-exceptions test.wat

In [49]:
runwasm("test.wasm")

<IPython.core.display.Javascript object>

If the index is valid, the output is `x[i]`

In [50]:
compileString("""
var x: [2 .. 4] → integer
program IndexOutOfBounds
    var i: integer
    x[2] := 2; x[3] := 3; x[4] := 4
    try
        i := 2
        write(x[i])
    catch 110
        write(110)
""", 'test.wat')

In [51]:
!wat2wasm --enable-exceptions test.wat

In [52]:
runwasm("test.wasm")

<IPython.core.display.Javascript object>